<a href="https://colab.research.google.com/github/tlsgptj/AIStudy/blob/main/2025_%EC%8B%A0%EC%95%BD%EA%B0%9C%EB%B0%9C_%EA%B2%BD%EC%A7%84%EB%8C%80%ED%9A%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rdkit

In [ ]:
pip install --upgrade xgboost

In [ ]:
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors, AllChem, MACCSkeys
from rdkit.ML.Descriptors import MoleculeDescriptors
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import xgboost as xgb

# 1. 데이터 로드
train = pd.read_csv("/content/drive/MyDrive/train.csv")[['Canonical_Smiles', 'Inhibition']]
test = pd.read_csv("/content/drive/MyDrive/test.csv")[['ID', 'Canonical_Smiles']]
submit = pd.read_csv("/content/drive/MyDrive/sample_submission.csv")

# 2. Descriptor 이름 및 계산기 정의
desc_names = [name for name, _ in Descriptors.descList]
desc_calc = MoleculeDescriptors.MolecularDescriptorCalculator(desc_names)

# 3. 특성 추출 함수
def extract_features(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol:
            # 1) RDKit Descriptors
            desc = np.array(desc_calc.CalcDescriptors(mol))

            # 2) Morgan Fingerprint
            morgan = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=2048)
            morgan = np.array(morgan)

            # 3) MACCS Keys
            maccs = np.array(MACCSkeys.GenMACCSKeys(mol))

            return np.concatenate([desc, morgan, maccs])
    except:
        return None
    return None

# 4. 피처 생성
train['Features'] = train['Canonical_Smiles'].apply(extract_features)
test['Features'] = test['Canonical_Smiles'].apply(extract_features)

# 5. 예외 제거
train = train[train['Features'].notnull()]
test = test[test['Features'].notnull()]

# 6. 배열 변환
train_x = np.stack(train['Features'].values)
train_y_raw = train['Inhibition'].astype(float).values
test_x = np.stack(test['Features'].values)

# ▶ log1p 적용 (선택)
use_log = True
if use_log:
    train_y = np.log1p(train_y_raw)
else:
    train_y = train_y_raw

# 7. 스케일링
scaler = StandardScaler()
train_x = scaler.fit_transform(train_x)
test_x = scaler.transform(test_x)

# 8. 데이터 분할
X_train, X_val, y_train, y_val = train_test_split(train_x, train_y, test_size=0.1, random_state=42)

# 9. 모델 정의
model = xgb.XGBRegressor(
    n_estimators=800,
    learning_rate=0.02,
    max_depth=7,
    subsample=0.85,
    colsample_bytree=0.85,
    reg_alpha=0.1,
    reg_lambda=1.0,
    random_state=42,
    tree_method='hist'  # GPU 사용 시 'gpu_hist'
)

model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    verbose=True
)

# 10. 예측 및 저장
test_pred = model.predict(test_x)
if use_log:
    submit['Inhibition'] = np.expm1(test_pred)  # 역변환
else:
    submit['Inhibition'] = test_pred

submit.to_csv("improved_submit.csv", index=False)
print("improved_submit.csv 저장 완료")


In [ ]:
pip install lightgbm

In [ ]:
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors, AllChem, MACCSkeys
from rdkit.ML.Descriptors import MoleculeDescriptors
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor

# 1. 데이터 로드
train = pd.read_csv("/content/drive/MyDrive/train.csv")[['Canonical_Smiles', 'Inhibition']]
test = pd.read_csv("/content/drive/MyDrive/test.csv")[['ID', 'Canonical_Smiles']]
submit = pd.read_csv("/content/drive/MyDrive/sample_submission.csv")

# 2. Descriptor 이름 및 계산기 정의
desc_names = [name for name, _ in Descriptors.descList]
desc_calc = MoleculeDescriptors.MolecularDescriptorCalculator(desc_names)

# 3. 특성 추출 함수
def extract_features(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol:
            desc = np.array(desc_calc.CalcDescriptors(mol))
            morgan = np.array(AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=2048))
            maccs = np.array(MACCSkeys.GenMACCSKeys(mol))
            return np.concatenate([desc, morgan, maccs])
    except:
        return None
    return None

# 4. 피처 생성
train['Features'] = train['Canonical_Smiles'].apply(extract_features)
test['Features'] = test['Canonical_Smiles'].apply(extract_features)

# 5. 예외 제거
train = train[train['Features'].notnull()]
test = test[test['Features'].notnull()]

# 6. 배열 변환
train_x = np.stack(train['Features'].values)
train_y_raw = train['Inhibition'].astype(float).values
test_x = np.stack(test['Features'].values)

# ▶ log1p 적용 여부
use_log = True
train_y = np.log1p(train_y_raw) if use_log else train_y_raw

# 7. 스케일링
scaler = StandardScaler()
train_x = scaler.fit_transform(train_x)
test_x = scaler.transform(test_x)

# 8. 데이터 분할
X_train, X_val, y_train, y_val = train_test_split(train_x, train_y, test_size=0.1, random_state=42)

# 9. 모델 정의 및 학습 (LightGBM)
model = CatBoostRegressor(
    iterations=2000,              # 더 많이 학습
    learning_rate=0.01,           # 더 작게 학습 (대신 iterations 증가)
    depth=8,                      # 복잡도 증가
    l2_leaf_reg=3,                # 정규화
    random_strength=1,           # 분할 다양성 증가
    loss_function='RMSE',
    eval_metric='RMSE',
    random_seed=42,
    verbose=100
)

model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)]
)

# 10. 예측 및 저장
test_pred = model.predict(test_x)
submit['Inhibition'] = np.expm1(test_pred) if use_log else test_pred
submit.to_csv("improved_submit_lgbm.csv", index=False)
print("improved_submit_lgbm.csv 저장 완료")


LigthBGM 0.5082516278
xgboost 0.587622083

LightGBM -> 모델에서 제외

In [ ]:
df_train = pd.read_csv("/content/drive/MyDrive/train.csv")

In [ ]:
df_train.head()

In [ ]:
df_test = pd.read_csv("/content/drive/MyDrive/test.csv")

In [ ]:
df_test.head()

## 앙상블 모델로 한번 가보자


In [ ]:
pip install catboost

In [ ]:
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors, AllChem, MACCSkeys
from rdkit.ML.Descriptors import MoleculeDescriptors
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor, Pool

# 1. 데이터 로드
train = pd.read_csv("/content/drive/MyDrive/train.csv")[['Canonical_Smiles', 'Inhibition']]
test = pd.read_csv("/content/drive/MyDrive/test.csv")[['ID', 'Canonical_Smiles']]
submit = pd.read_csv("/content/drive/MyDrive/sample_submission.csv")

# 2. Descriptor 이름 및 계산기 정의
desc_names = [name for name, _ in Descriptors.descList]
desc_calc = MoleculeDescriptors.MolecularDescriptorCalculator(desc_names)

# 3. 특성 추출 함수
def extract_features(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol:
            desc = np.array(desc_calc.CalcDescriptors(mol))
            morgan = np.array(AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=2048))
            maccs = np.array(MACCSkeys.GenMACCSKeys(mol))
            return np.concatenate([desc, morgan, maccs])
    except:
        return None
    return None

# 4. 피처 생성
train['Features'] = train['Canonical_Smiles'].apply(extract_features)
test['Features'] = test['Canonical_Smiles'].apply(extract_features)

# 5. 예외 제거
train = train[train['Features'].notnull()]
test = test[test['Features'].notnull()]

# 6. 배열 변환
train_x = np.stack(train['Features'].values)
train_y_raw = train['Inhibition'].astype(float).values
test_x = np.stack(test['Features'].values)

# ▶ log1p 적용 여부
use_log = True
train_y = np.log1p(train_y_raw) if use_log else train_y_raw

# 7. 스케일링
scaler = StandardScaler()
train_x = scaler.fit_transform(train_x)
test_x = scaler.transform(test_x)

# 8. 데이터 분할
X_train, X_val, y_train, y_val = train_test_split(train_x, train_y, test_size=0.1, random_state=42)

# 9. CatBoost 모델 정의
model = CatBoostRegressor(
    iterations=1000,
    learning_rate=0.03,
    depth=7,
    loss_function='RMSE',
    verbose=100,
    random_seed=42
)

# 10. 학습
model.fit(X_train, y_train, eval_set=(X_val, y_val))

# 11. 예측 및 저장
test_pred = model.predict(test_x)
submit['Inhibition'] = np.expm1(test_pred) if use_log else test_pred
submit.to_csv("improved_submit_catboost.csv", index=False)
print("improved_submit_catboost.csv 저장 완료")


catboost -> 0.68612 현재 가장 데이터와 잘 붙는 모델

## 왜 Catboost가 가장 성능이 좋았을까?

- 범용성 좋은 결정트리 기반 부스팅 : 범주형 데이터와 연속형 모두에 강한 모델, 과적합 방지와 일반화 성능에 특히 뛰어남

- 자동 순서 샘플링 & 편향 보정 : Catboost 고유의 기술로 트리 학습 시 편향을 줄여 예측력이 높음

- 스케일링 불필요 -> 데이터 특성 유지 : 내부적으로 스케일링이 없이도 안정적으로 학습이 가능함, 화학 구조 특징 등 복잡한 데이터에서 유리함

- 빠른 수렴 및 최적화 : 적은 파라미터 조정으로도 빠르게 좋은 결과를 도출 가능함

Catboost 코드 분석

1. 데이터 로드 : RDkit 라이브러리로 화학 구조 SMILES-> 물리화학 지표, MACCS 키 추출

2. SMILES 특성 백터 생성

3. 결측치나 오류 데이터는 제외

4. 모델 입력용 배열화, Inhibition 값 로그 변환 여부 선택

5. StandardScaler로 평균 0, 분산 1 정규화

6. 학습

RDkit란? 화학 정보 처리를 위한 오픈소스 파이썬 라이브러리

- 화학 분자 구조 다루기 (SMILES, Mol 파일 등 읽기/쓰기)

- 분자 특성(Descriptors) 계산: 분자량, 극성, 지문(Fingerprint) 등

- 분자 변환 및 조작 (예: 구조 변경, 합성 가능성 검사)

- 분자 유사도 계산 및 검색

- 화합물 데이터 전처리 및 머신러닝용 특징 벡터 생성

## 같은 모델 여러 개 → 평균 = Bagging 계열 앙상블

In [ ]:
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors, AllChem, MACCSkeys
from rdkit.ML.Descriptors import MoleculeDescriptors
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor

# 1. 데이터 로드
train = pd.read_csv("/content/drive/MyDrive/train.csv")[['Canonical_Smiles', 'Inhibition']]
test = pd.read_csv("/content/drive/MyDrive/test.csv")[['ID', 'Canonical_Smiles']]
submit = pd.read_csv("/content/drive/MyDrive/sample_submission.csv")

# 2. Descriptor 계산기 정의
desc_names = [name for name, _ in Descriptors.descList]
desc_calc = MoleculeDescriptors.MolecularDescriptorCalculator(desc_names)

# 3. 특성 추출 함수
def extract_features(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol:
            desc = np.array(desc_calc.CalcDescriptors(mol))
            morgan = np.array(AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=2048))
            maccs = np.array(MACCSkeys.GenMACCSKeys(mol))
            return np.concatenate([desc, morgan, maccs])
    except:
        return None
    return None

# 4. 피처 생성
train['Features'] = train['Canonical_Smiles'].apply(extract_features)
test['Features'] = test['Canonical_Smiles'].apply(extract_features)

# 5. 예외 제거
train = train[train['Features'].notnull()]
test = test[test['Features'].notnull()]

# 6. 배열 변환
train_x = np.stack(train['Features'].values)
train_y_raw = train['Inhibition'].astype(float).values
test_x = np.stack(test['Features'].values)

# ▶ log1p 적용 여부
use_log = True
train_y = np.log1p(train_y_raw) if use_log else train_y_raw

# 7. 스케일링
scaler = StandardScaler()
train_x = scaler.fit_transform(train_x)
test_x = scaler.transform(test_x)

# 8. 앙상블 학습 및 예측
n_models = 5
preds = []

for seed in range(n_models):
    # 데이터 분할
    X_train, X_val, y_train, y_val = train_test_split(
        train_x, train_y, test_size=0.1, random_state=42 + seed
    )

    # CatBoost 모델 정의
    model = CatBoostRegressor(
        iterations=1000,
        learning_rate=0.03,
        depth=7,
        loss_function='RMSE',
        random_seed=42 + seed,
        verbose=100
    )

    # 모델 학습
    model.fit(X_train, y_train, eval_set=(X_val, y_val), verbose=100)

    # 예측 저장
    pred = model.predict(test_x)
    preds.append(pred)

# 9. 예측 평균
ensemble_pred = np.mean(preds, axis=0)
submit['Inhibition'] = np.expm1(ensemble_pred) if use_log else ensemble_pred

# 10. 결과 저장
submit.to_csv("ensemble_submit_catboost.csv", index=False)
print("ensemble_submit_catboost.csv 저장 완료")

## 0.6849 -> 2번째로 높음

## 다른 모델 결합

In [ ]:
pip install rdkit-pypi catboost xgboost lightgbm

In [ ]:
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors, AllChem, MACCSkeys
from rdkit.ML.Descriptors import MoleculeDescriptors
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
import xgboost as xgb
from lightgbm import LGBMRegressor
import lightgbm as lgb  # callbacks 위해 별도 import

# 1. 데이터 로드
train = pd.read_csv("/content/drive/MyDrive/train.csv")[['Canonical_Smiles', 'Inhibition']]
test = pd.read_csv("/content/drive/MyDrive/test.csv")[['ID', 'Canonical_Smiles']]
submit = pd.read_csv("/content/drive/MyDrive/sample_submission.csv")

# 2. Descriptor 계산기 정의
desc_names = [name for name, _ in Descriptors.descList]
desc_calc = MoleculeDescriptors.MolecularDescriptorCalculator(desc_names)

# 3. 특성 추출 함수
def extract_features(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol:
            desc = np.array(desc_calc.CalcDescriptors(mol))
            morgan = np.array(AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=2048))
            maccs = np.array(MACCSkeys.GenMACCSKeys(mol))
            return np.concatenate([desc, morgan, maccs])
    except:
        return None
    return None

# 4. 피처 생성
train['Features'] = train['Canonical_Smiles'].apply(extract_features)
test['Features'] = test['Canonical_Smiles'].apply(extract_features)

# 5. 예외 제거
train = train[train['Features'].notnull()]
test = test[test['Features'].notnull()]

# 6. 배열 변환
train_x = np.stack(train['Features'].values)
train_y_raw = train['Inhibition'].astype(float).values
test_x = np.stack(test['Features'].values)

# 7. log1p 변환
use_log = True
train_y = np.log1p(train_y_raw) if use_log else train_y_raw

# 8. 스케일링
scaler = StandardScaler()
train_x = scaler.fit_transform(train_x)
test_x = scaler.transform(test_x)

# 9. 데이터 분할
X_train, X_val, y_train, y_val = train_test_split(
    train_x, train_y, test_size=0.1, random_state=42
)

# 10. CatBoost 모델 학습
cat_model = CatBoostRegressor(
    iterations=1000,
    learning_rate=0.03,
    depth=7,
    loss_function='RMSE',
    random_seed=42,
    verbose=100
)
cat_model.fit(X_train, y_train, eval_set=(X_val, y_val), verbose=100)
cat_pred = cat_model.predict(test_x)

# 11. XGBoost 모델 학습 (구버전 호환)
dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)
dtest = xgb.DMatrix(test_x)

params = {
    'objective': 'reg:squarederror',
    'learning_rate': 0.03,
    'max_depth': 7,
    'seed': 42
}
evals = [(dval, 'eval')]

xgb_model = xgb.train(
    params,
    dtrain,
    num_boost_round=1000,
    evals=evals,
    early_stopping_rounds=50,
    verbose_eval=100
)

xgb_pred = xgb_model.predict(dtest)

# 12. LightGBM 모델 학습 (callbacks 사용)
lgb_model = LGBMRegressor(
    n_estimators=1000,
    learning_rate=0.03,
    max_depth=7,
    objective='regression',
    random_state=42
)
lgb_model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    callbacks=[
        lgb.early_stopping(stopping_rounds=50),
        lgb.log_evaluation(period=100)
    ]
)
lgb_pred = lgb_model.predict(test_x)

# 13. 예측 평균 (앙상블)
ensemble_pred = (cat_pred + xgb_pred + lgb_pred) / 3
submit['Inhibition'] = np.expm1(ensemble_pred) if use_log else ensemble_pred

# 14. 결과 저장
submit.to_csv("ensemble_submit_all_models.csv", index=False)
print("ensemble_submit_all_models.csv 저장 완료")

## 0.62로 성능이 더 낮아짐 why?

In [ ]:
pip install numpy==1.23.5

In [ ]:
pip install --upgrade catboost xgboost lightgbm